Name - Shashank Singh
Roll number - 19289
Department - Data Science Engineering

1. Adding the required libraries to the environment 

In [1]:
#pip install dash

In [2]:
#pip install flask

In [3]:
#pip install dash-bootstrap-components

In [4]:
#pip install alpha-vantage

Importing the required libraries

In [5]:
import flask
import dash                                # pip install dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from alpha_vantage.timeseries import TimeSeries

2. This shell needs to be run only once, to obtain the dataframe in the CSV format from the API.

In [6]:
# -------------------------------------------------------------------------------
# Set up initial key and financial category

# key = '9UG6E9D0IU0ZWCYK' # Using the key obtained for acessing the alpha vantage API. 

# # # Choosing pandas as the output format

# ts = TimeSeries(key, output_format='pandas') # 'pandas'
# aapl_data, aapl_meta_data = ts.get_intraday(symbol='AAPL',interval='1min', outputsize='compact') #picking up the data of share price for Apple inc.
# df = aapl_data.copy()
# df=df.transpose()
# df.rename(index={"1. open":"open", "2. high":"high", "3. low":"low",
#                  "4. close":"close","5. volume":"volume"},inplace=True)
# df=df.reset_index().rename(columns={'index': 'indicator'})
# df = pd.melt(df,id_vars=['indicator'],var_name='date',value_name='rate')
# df = df[df['indicator']!='volume']
# print(df.head(10))


# df.to_csv("data2.csv", index=False) #data2 shall represent the data obtained from the API
# exit()

#this needs to run only once to extract the data from the API in the CSV format

3. Looking at the data.

In [7]:
dff = pd.read_csv("data2.csv")
dff = dff[dff.indicator.isin(['high'])] #Taking only the 'high' values from the dataset

In [8]:
dff.head() #peeking into the dataset

,indicator,date,rate
1,high,2021-12-03 20:00:00,161.34
5,high,2021-12-03 19:59:00,161.35
9,high,2021-12-03 19:58:00,161.32
13,high,2021-12-03 19:57:00,161.40
17,high,2021-12-03 19:56:00,161.43


4. Building the dashboard.

The layout of the dashboard will be in the form of a card that will have 5 Rows and maximum two column each.

In [9]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]
                )


app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Card(
                [
                    dbc.CardImg(
                        src="/assets/logo.png",
                        top=True,
                        style={"width": "8rem"},
                        className="ml-3"),
                    dbc.CardBody([
                        
                        #Row 1
                        ##this row will have the name of the stock.
                        
                        dbc.Row([
                            dbc.Col([
                                html.P("AAPL", className="ml-3")
                            ],width={'size':5, 'offset':1}),

                        ]),
                        
                        #Row 2
                        ##this row will indicate the change in stock price
                        
                        dbc.Row([
                            dbc.Col([
                                html.P("CHANGE (1D)", className="ml-3")
                            ],width={'size':5, 'offset':1}),

                            dbc.Col([
                                dcc.Graph(id='indicator-graph', figure={},
                                          config={'displayModeBar':False},
                                          )
                            ], width={'size':3,'offset':2})
                        ]),
                        
                        #Row 3
                        #this row shall have the plot for the variation in price.

                        dbc.Row([
                            dbc.Col([
                                dcc.Graph(id='daily-line', figure={},
                                          config={'displayModeBar':False})
                            ], width=12)
                        ]),

                        #Row 4
                        #This row will have the buttons with the buy and sell option.
                        
                        dbc.Row([
                            dbc.Col([
                                dbc.Button("SELL", className="ml-5"),
                            ], width=4),

                            dbc.Col([
                                dbc.Button("BUY")
                            ], width=4)
                        ], justify="between"),
                        
                        #Row 5
                        #This row shall show the current pricing.

                        dbc.Row([
                            dbc.Col([
                                dbc.Label(id='low-price', children="12.237",
                                          className="mt-2 ml-5 bg-white p-1 border border-primary border-top-0"),
                            ],width=4),
                            dbc.Col([
                                dbc.Label(id='high-price',
                                          className="mt-2 bg-white p-1 border border-primary border-top-0"),
                            ], width=4)
                        ], justify="between")
                    ]),
                ],
                style={"width": "24rem"},
                className="mt-3"
            )
        ], width=6)
    ], justify='center'),

    dcc.Interval(id='update', n_intervals=0, interval=1000*5) #interval used to update the app every 500 seconds
])


5. Adding more features to the dashboard.

In [10]:

# Indicator Graph-----------------------------------------------------------


@app.callback(
    Output('indicator-graph', 'figure'),
    Input('update', 'n_intervals')
)
def update_graph(timer):
    dff_rv = dff.iloc[::-1]
    day_start = dff_rv[dff_rv['date'] == dff_rv['date'].min()]['rate'].values[0]
    day_end = dff_rv[dff_rv['date'] == dff_rv['date'].max()]['rate'].values[0]

    fig = go.Figure(go.Indicator(
        mode="delta",
        value=day_end,
        delta={'reference': day_start, 'relative': True, 'valueformat':'.2%'}))
    fig.update_traces(delta_font={'size':12})
    fig.update_layout(height=30, width=70)

    if day_end >= day_start:
        fig.update_traces(delta_increasing_color='green')
    elif day_end < day_start:
        fig.update_traces(delta_decreasing_color='red')

    return fig


# Line Graph---------------------------------------------------------------
@app.callback(
    Output('daily-line', 'figure'),
    Input('update', 'n_intervals')
)
def update_graph(timer):
    dff_rv = dff.iloc[::-1]
    fig = px.line(dff_rv, x='date', y='rate',
                   range_y=[dff_rv['rate'].min(), dff_rv['rate'].max()],
                   height=120).update_layout(margin=dict(t=0, r=0, l=0, b=20),
                                             paper_bgcolor='rgba(0,0,0,0)',
                                             plot_bgcolor='rgba(0,0,0,0)',
                                             yaxis=dict(
                                             title=None,
                                             showgrid=False,
                                             showticklabels=False
                                             ),
                                             xaxis=dict(
                                             title=None,
                                             showgrid=False,
                                             showticklabels=False
                                             ))

    day_start = dff_rv[dff_rv['date'] == dff_rv['date'].min()]['rate'].values[0]
    day_end = dff_rv[dff_rv['date'] == dff_rv['date'].max()]['rate'].values[0]

    if day_end >= day_start:
        return fig.update_traces(fill='tozeroy',line={'color':'green'})
    elif day_end < day_start:
        return fig.update_traces(fill='tozeroy',line={'color': 'red'})
    
    
    
# Below the buttons--------------------------------------------------------
# This section styles the part of the dashboard below the button.

@app.callback(
    Output('high-price', 'children'),
    Output('high-price', 'className'),
    Input('update', 'n_intervals')
)
def update_graph(timer):
    if timer ==0:
        dff_filtered = dff.iloc[[21,22]]
        print(dff_filtered)
    elif timer == 1:
        dff_filtered = dff.iloc[[20,21]]
        print(dff_filtered)
    elif timer == 2:
        dff_filtered = dff.iloc[[19,20]]
        print(dff_filtered)
    elif timer == 3:
        dff_filtered = dff.iloc[[18,19]]
        print(dff_filtered)
    elif timer == 4:
        dff_filtered = dff.iloc[[17,18]]
        print(dff_filtered)
    elif timer == 5:
        dff_filtered = dff.iloc[[16,17]]
        print(dff_filtered)
    elif timer > 5:
        return dash.no_update

    recent_high = dff_filtered['rate'].iloc[0]
    older_high = dff_filtered['rate'].iloc[1]

#the following block of codes decide when the colour of the plot changes.

    if recent_high > older_high:
        return recent_high, "mt-2 bg-success text-white p-1 border border-primary border-top-0"
    elif recent_high == older_high:
        return recent_high, "mt-2 bg-white p-1 border border-primary border-top-0"
    elif recent_high < older_high:
        return recent_high, "mt-2 bg-danger text-white p-1 border border-primary border-top-0"
    

if __name__=='__main__':
    app.run_server(debug = False, port=8000)

Dash is running on http://127.0.0.1:8000/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


OSError: [Errno 48] Address already in use